# _Scala Fundamentals_
# 6. Case Classes, Pattern Matching, and Algebraic Data Types

_This module was originally developed by Michael Pilquist._

## Case Classes: The gist

The `case` modifier on a class results in:

- Automatic `val` for each constructor parameter
- Structural `equals` and `hashCode`
- Structural `toString`
- Automatically generated companion object (with an `apply` method, so you can instantiate without using `new`)

  Case classes are still classes though! You can add methods, values, etc.

In [ ]:
case class PhoneNumber(areaCode: String, number: String)

case class Person(firstName: String, 
                  lastName: String, 
                  salary: Int, 
                  ssn: String,
                  phoneNumber: Option[PhoneNumber]) {
  def fullName: String = firstName + " " + lastName
}

val AllPeople = List(
  Person("Lavern", "Clyatt", 203190, "913-98-6520", Some(PhoneNumber("215", "555-1212"))),
  Person("Dia", "Unverzagt", 57945, "955-66-1239", Some(PhoneNumber("610", "555-1313"))),
  Person("Madison", "Vaszily", 107455, "942-90-6988", Some(PhoneNumber("267", "555-1212"))),
  Person("Elwanda", "Hulse", 207472, "978-38-9209", None),
  Person("Mina", "Suckow", 231743, "943-68-7575", None)
)

val lavern: Person = AllPeople(0)
println(s"Lavern's last name is ${lavern.lastName}")
println(s"Lavern's SSN is ${lavern.ssn}")
println(s"Lavern's full name is ${lavern.fullName}")
println(s"Lavern: $lavern")


val dia: Person = AllPeople(1)

println(s"Lavern == Dia? ${lavern == dia}")
println(s"Lavern == Lavern? ${lavern == lavern}")

%md

![?](http://i.imgur.com/Guv4TBn.png) **So what? I could have written this myself!**

- Lots and lots of boilerplate
- Generally accepted best practice to use a case class any time we want `equals`/`hashCode`/`toString` for a data struture
- Encourages immutability and simple data types

Here's the non-case class version

In [ ]:
class Person2(val firstName: String,
              val lastName: String, 
              val salary: Int, 
              val ssn: String, 
              val phoneNumber: Option[PhoneNumber]) {
  override def toString: String = s"String($firstName,$lastName,$salary,$ssn,$phoneNumber)"
  
  // Don't worry about match. We'll get to that soon.
  override def equals(o: Any): Boolean = o match {
    case that: Person2 => 
      firstName == that.firstName && 
      lastName == that.lastName && 
      salary == that.salary && 
      ssn == that.ssn &&
      phoneNumber == that.phoneNumber
    case _ => false
  }
  
  override def hashCode: Int = ??? // Please help me!
}

object Person2 {
  def apply(firstName: String, 
            lastName: String, 
            salary: Int, 
            ssn: String, 
            phoneNumber: Option[PhoneNumber]): Person2 = {
    new Person2(firstName, lastName, salary, ssn, phoneNumber)
  }
}

## Copying Case Classes

Every case class has a `copy` method which returns a clone of the case class with selective changes.

In [ ]:
val shirley = lavern.copy(firstName = "Shirley", lastName = "Feeney")
val noche = dia.copy(firstName = "Noche", ssn = "123-45-6789")

%md

![?](http://i.imgur.com/Guv4TBn.png) **How would we implement this without case classes?**

- Default parameters initialized to field values
- Named parameters at call site

In [ ]:
class Person2(val firstName: String,
              val lastName: String, 
              val salary: Int, 
              val ssn: String, 
              val phoneNumber: Option[PhoneNumber]) {
  override def toString: String = s"String($firstName,$lastName,$salary,$ssn,$phoneNumber)"
  override def equals(o: Any): Boolean = ???
  override def hashCode: Int = ???
  
  def copy(firstName: String = this.firstName, 
           lastName: String = this.lastName,
           salary: Int = this.salary,
           ssn: String = this.ssn,
           phoneNumber: Option[PhoneNumber] = this.phoneNumber): Person2 = {
    new Person2(firstName, lastName, salary, ssn, phoneNumber)
  }

}

object Person2 {
  def apply(firstName: String, 
            lastName: String, 
            salary: Int, 
            ssn: String, 
            phoneNumber: Option[PhoneNumber]): Person2 = {
    new Person2(firstName, lastName, salary, ssn, phoneNumber)
  }
}

val lavern2 = Person2("Lavern", "Clyatt", 203190, "913-98-6520", Some(PhoneNumber("215", "555-1212")))
val shirley2 = lavern.copy(firstName = "Shirley", lastName = "Feeney")

## Exercise 6.1

- Use a case class to model a bank account, which consists of an account holder (`Person`) and a balance in dollars. 
- Define a method which debits the account by a specified number of dollars, returning a copy of the account with the adjusted balance.
- Define a method to update the phone number, returning a copy of the account with the new phone number.

In [ ]:
// Put your solution here

In [ ]:
// ANSWER
// This one's better.

case class Account(holder: Person, balance: Int) {
  
  def debit(amount: Int): Account = {
    copy(balance = balance - amount)
  }

  def updatePhoneNumber(ph: PhoneNumber): Account = {
    copy(holder = holder.copy(phoneNumber = Some(ph)))
  }
}

## Validation

How can we validate the values in a case class? Two insurmountable issues:

- The generated `apply` method in the companion _cannot_ be overridden or customized.
- The `copy` method allows insertion of arbitrary values.

One option is using the `require` method, or otherwise throwing exceptions, from case class constructor:

In [ ]:
case class Phone(areaCode: String, number: String) {
  require(areaCode.size == 3, "area code must be 3 digits")
  require(number.size == 8, "number must be in the format xxx-xxxx")
}

val p = Phone("215", "555-1212")
scala.util.Try(p.copy(areaCode = "21"))

This leaves a lot to be desired though, as we might want to perform validation that's safe — i.e., that doesn't throw exceptions.

To accomplish this, we can:

- declare the case class *abstract*, which suppresses the creation of the `apply` method in the companion and the `copy` method in the class
- declare the class *sealed* so that no subtype can be defined outside of the file in which the abstract class is defined
- implement our own `apply` method that returns a private subtype

**Note**: The `Either` type is a way to capturing and returning one of two possible values. It's often used for
error handling. `Either[String, Int]` can return a `Left(string)` or a `Right(integer)`. Conventionally, we use
the `Left` type to hold an error and the `Right` type to hold the successful return value.

In [ ]:
sealed abstract case class Phone(areaCode: String, number: String)

object Phone {
  def apply(areaCode: String, number: String): Either[String, Phone] = {
    if (areaCode.size != 3) Left("area code must be 3 digits")
    else if (number.size != 8) Left("number must be in the format xxx-xxxx")
    else Right(new Phone(areaCode, number) {})
  }
}
println(Phone("215", "555-1212"))
println(Phone("21", "555-1212"))
println(Phone("215", "555-1212") == Phone("215", "555-1212"))

In [ ]:
val ph = Phone("610", "555-1212").right.get // this is kind of dangerous

# Pattern Matching

The `match` and `case` keywords let us inspect an expression using various patterns or conditions. In its simplest form, this is analogous to `switch`/`case` from C or Java.

In [ ]:
def showDirection(dir: Int): String = {
  dir match {
    case 1 => "East"
    case 2 => "North"
    case 3 => "West"
    case 4 => "South"
  }
}

println(showDirection(1))
println(showDirection(2))
println(showDirection(3))
println(showDirection(4))

If the expression does not match any of the cases, a `scala.MatchError` is thrown:

In [ ]:
try {
  showDirection(5)
}
catch {
  case e: Exception => println(s"${e.getClass.getName}: ${e.getMessage}")
}

We can add a default case by pattern-matching on (what else?) an `_`:

In [ ]:
def showDirection(dir: Int): String = {
  dir match {
    case 1 => "East"
    case 2 => "North"
    case 3 => "West"
    case 4 => "South"
    case _ => "Unknown"
  }
}
println(showDirection(5))


### Destructuring

Pattern matching is much more powerful than simple switch constructs. The biggest enabler is *destructuring*, which lets us:

- "take apart" a value based on some pattern, and
- bind names to the constituent parts.

We can destructure tuples by pattern matching with their "shape":

In [ ]:
def sumComponents(p: (Int, Int, Int)): Int = {
  p match {
    case (x, y, z) => x + y + z
  }
}

sumComponents((1, 2, 3))

### Subtype Matching

We can also pattern match on subtypes.

Recall that `Option` is defined with two subtypes: `Some(value)` and `None`. We can use this to pattern match on options:

In [ ]:
def square(x: Option[Int]): Option[Int] = {
  x match {
    case s: Some[Int] => Some(s.get * s.get) // Eek! There's a better way to do this...
    case None         => None
  }
}

println(square(Some(5)))
println(square(None))

When you find yourself calling `get` on an `Option`, you're usually doing it wrong.

We can actually destructure the `Some` using a constructor-like pattern:

In [ ]:
def square(x: Option[Int]): Option[Int] = {
  x match {
    case Some(xx) => Some(xx * xx)
    case None     => None
  }
}

println(square(Some(5)))
println(square(None))

Of course, the _functional_ way to implement `square` is with `map`:

In [ ]:
def square(x: Option[Int]): Option[Int] = x.map { i => i * i }

println(square(Some(5)))
println(square(None))

### Exercise 6.2: Implement sum of squares using a single match expression

Implement a function called `sumOfSquares`.

- The implementation should _not_ use any functions on `Option` like `map` or `getOrElse`.
- The implementation can be done with a single `match` statement.

This one's a little tricky. **Hint**: Consider wrapping the arguments in a 2-tuple.


In [ ]:
// Fill in the following function, replacing None with an implementation that prints true for all 4 test cases.

def sumOfSquares(x: Option[Int], y: Option[Int]): Option[Int] = {
  None
}

println(sumOfSquares(Some(5), Some(4)) == Some(41))
println(sumOfSquares(Some(5), None) == None)
println(sumOfSquares(None, Some(4)) == None)
println(sumOfSquares(None, None) == None)

In [ ]:
// ANSWER
def sumOfSquares(x: Option[Int], y: Option[Int]): Option[Int] = {
  (x, y) match {
    case (Some(xx), Some(yy)) => Some(xx * xx + yy * yy)
    case _ => None
  }
}

assert(sumOfSquares(Some(5), Some(4)) == Some(41))
assert(sumOfSquares(Some(5), None) == None)
assert(sumOfSquares(None, Some(4)) == None)
assert(sumOfSquares(None, None) == None)

# Pattern Matching on Case Classes

- Case classes support destructuring based on their fields
- Each field can either by bound to a variable or further matched

**Trivia**: They're called "case classes" because one of their original purposes was to facilitate pattern ("case") matching.

In [ ]:
def lastNameAndAreaCode(p: Person): Option[(String, String)] = {
  p match {
    case Person(firstName, lastName, salary, ssn, Some(PhoneNumber(areaCode, digits))) =>
      Some((lastName, areaCode))
    case _ => None
  }
}

AllPeople.map(lastNameAndAreaCode)

- We didn't use many of the fields of the case class but we still need to match on them. 
- When we destructure a case class, we need to exactly match the structure.
- We *don't* need to bind names to the values we aren't going to use! 

Let's call on our old, abused friend, the underscore:

In [ ]:
def lastNameAndAreaCode(p: Person): Option[(String, String)] = {
  p match {
    case Person(_, lastName, _, _, Some(PhoneNumber(areaCode, _))) =>
      Some((lastName, areaCode))
    case _ => None
  }
}

AllPeople.map(lastNameAndAreaCode)

## Guards

Each case statement can optionally have a _guard_ — an `if booleanExpression` which further limits the values the case succeeds on.

**Oddity:** We don't *need* parentheses around the boolean expression. This is a syntax quirk of Scala!

In [ ]:
def migratePhones(p: Person): Person = {
  p match {
    case Person(_, lastName, _, _, Some(PhoneNumber("215", digits))) if digits.startsWith("555-") =>
      p.copy(phoneNumber = Some(PhoneNumber("267", digits)))
    case _ => p
  }
}

AllPeople.map(migratePhones)

## Pattern Matching Syntax

We've seen a lot of name bindings in patterns. For example, 

```scala
case Person(_, lastName, _, _, Some(PhoneNumber("215", digits)))
```

contains two name bindings, `lastName` and `digits`, along with three ignored patterns (the underscores), and one literal pattern (the `"215"`).

Every pattern supports name binding though! The [Scala Language Specification (section 8.1)](http://scala-lang.org/files/archive/spec/2.11/08-pattern-matching.html) defines pattern matching syntax in the following way (using an EBNF-like syntax):

```
Pattern         ::=  Pattern1 { ‘|’ Pattern1 }
Pattern1        ::=  varid ‘:’ TypePat
                  |  ‘_’ ‘:’ TypePat
                  |  Pattern2
Pattern2        ::=  varid [‘@’ Pattern3]
                  |  Pattern3
Pattern3        ::=  SimplePattern
                  |  SimplePattern {id [nl] SimplePattern}
SimplePattern   ::=  ‘_’
                  |  varid
                  |  Literal
                  |  StableId
                  |  StableId ‘(’ [Patterns] ‘)’
                  |  StableId ‘(’ [Patterns ‘,’] [varid ‘@’] ‘_’ ‘*’ ‘)’
                  |  ‘(’ [Patterns] ‘)’
                  |  XmlPattern
Patterns        ::=  Pattern {‘,’ Patterns}
```

Aha! This definition is illuminating in a number of ways:
 - Patterns are *recursive*
 - Name bindings are supported in mutiple ways
 - Pattern matching syntax does not say anything about case classes

Let's look at each of these in detail.

### Patterns are recursive

To get a feel for what this means, let's take a pattern we used earlier and project it against the syntax definition from the SLS:

```scala
Person(firstName, lastName, salary, ssn, Some(PhoneNumber(areaCode, digits)))
```

We start with the token `Person` and with the `Pattern` definition. 

- `Person` must be a `Pattern1`. 
- Looking at `Pattern1`, `Person` isn't a `varid : TypePat` nor a `_: TypePat` so it must be a `Pattern2`. 
- `Person` isn't a `varid @ Pattern3` so it must be a `SimplePattern`. 
- It's in the definition of `SimplePattern` where we make progress: `Person` is a *stable identifier* and it is followed by parentheses enclosing more patterns. 

This gives us the following traversal of grammar rules:

```
Pattern -> Pattern1 -> Pattern2 -> Pattern3 -> SimplePattern -> StableId '(' [Patterns] ')'
```

The `Patterns` rule expands to a comma separate list of `Pattern`, which means we can parse each argument to `Person` as a pattern. For example, parsing `firstName` is accomplished via the traversal `Pattern -> Pattern1 -> Pattern2 -> varid`.

Why does this matter to us? Because we can use full pattern syntax at each "level" of destructuring!

In [ ]:
def foo(o: Option[Person]) = o match {
  case Some(p @ Person("Lavern", _, _, _, Some(ph))) if ph.number.endsWith("1212") =>
    println("Full name: " + p.fullName)
  case other => "Doesn't match!"
}
foo(Some(lavern))

### Name bindings are supported in multiple ways

What we've been referring to as a "name binding" is represented in the grammar as productions which use the `varid` rule. These are:
```
Pattern1        ::=  varid ‘:’ TypePat | ...
Pattern2        ::=  varid [‘@’ Pattern3] | Pattern3
SimplePattern   ::=  ‘_’ | varid | ...
```

A pattern of the form `varid: TypePat` is a *typed pattern*. Like we saw before, it matches when the expression is assignable to the type specified by `TypePat` and it introduces a new value in to scope with the name `varid`.

A pattern of the form `varid @ Pattern` is a *pattern binder*. It introduces a new value in to scope with the name `varid` such that the value bound to `varid` matches the pattern after the `@`.

### Pattern matching syntax is not case class aware (or is it?)

In the syntax definition a pattern match, case classes were not mentioned. When we match on case classes, we use the grammar rule:

```
SimplePattern -> StableId ‘(’ [Patterns] ‘)’
```

Scala uses this grammar rule for two purposes: *constructor patterns* and *extractor patterns*.

- A constructor pattern is a pattern which matches the primary constructor of a case class — i.e., a pattern is 
  specified inside the parentheses for each argument to the primary constructor of the case class.
- An extractor pattern is a pattern where the `StableId` refers to an object that has one or more methods named `unapply`. We'll see these more in a bit.

## Stable Identifiers

In [ ]:
def inPhilly(ph: PhoneNumber): Boolean = ph match { case PhoneNumber("215", _) => true; case _ => false }
println(inPhilly(PhoneNumber("215", "555-1212")))
println(inPhilly(PhoneNumber("866", "555-1313")))

Let's define `"215"` as a constant instead of hard-coding it in our pattern:

In [ ]:
val primaryPhillyExchange = "215"
def inPhilly(ph: PhoneNumber): Boolean = ph match { case PhoneNumber(primaryPhillyExchange, _) => true; case _ => false }
println(inPhilly(PhoneNumber("215", "555-1212")))
println(inPhilly(PhoneNumber("866", "555-1313")))

![!](http://i.imgur.com/TvgcpDH.png)
Eek! What's happening here? 

The `PhoneNumber(primaryPhillyExchange, _)` pattern is getting parsed as a constructor pattern with the two patterns:

- `primaryPhillyExchange` and 
- `_`. 

The `primaryPhillyExchange` pattern is getting parsed as `Pattern -> Pattern1 -> Pattern2 -> varid`. That is, `primaryPhillyExchange` is acting as a name binding.

Scala defines `varid` as identifiers that start with a lowercase character. We can take advantage of this by naming our constant to start with an uppercase character.

In [ ]:
val PrimaryPhillyExchange = "215"
def inPhilly(ph: PhoneNumber): Boolean = ph match { case PhoneNumber(PrimaryPhillyExchange, _) => true; case _ => false }
println(inPhilly(PhoneNumber("215", "555-1212")))
println(inPhilly(PhoneNumber("866", "555-1313")))

## Alternative Patterns

Scala also supports "OR"-ing patterns using the `|` character to separate patterns.

In [ ]:
def inPhilly(ph: PhoneNumber): Boolean = ph match { 
  // We're probably missing a few...
  case PhoneNumber("215", _) | PhoneNumber("267", _) | PhoneNumber("610", _) | PhoneNumber("484", _) => true
  case _ => false
}

println(inPhilly(PhoneNumber("215", "555-1212")))
println(inPhilly(PhoneNumber("267", "555-1313")))
println(inPhilly(PhoneNumber("610", "555-1414")))

def foo(x: Int): String = x match { 
  case -1 | 0 | 1 => "Nearly Zero"
  case _ => "Not Nearly Zero" 
}

println(foo(1))
println(foo(4))

We cannot use alternatives and name bindings in the same pattern though. For example, this does not compile:


In [ ]:
// This won't compile. (Try it.)

def foo(ph: PhoneNumber) = ph match { case PhoneNumber("215", digits) | PhoneNumber("267", digits) => println(digits) }

We can use name bindings before we get to the alternative pattern though — for example, by using a pattern binder:

In [ ]:
def foo(ph: PhoneNumber) = ph match { case p @ (PhoneNumber("215", _) | PhoneNumber("267", _)) => println(p.number) }

## Extractors

When destructuring a case class with a pattern match, we used the grammar production:

```
SimplePattern -> StableId ‘(’ [Patterns] ‘)’
```

More specifically, we used a constructor pattern, which destructured the case class by matching each argument to an argument in the primary constructor.

This same grammar production supports *extractor patterns*. The `StableId` is the name of an object that defines an `unapply` method. The `unapply` method takes apart a value, returning its pieces.

In [ ]:
object SocialSecurityNumber {
  def unapply(str: String): Option[(Int, Int, Int)] = {
    val parts = str.split("-")
    if (parts.size == 3) {
      try {
        val nums = parts.map(_.toInt)
        Some((nums(0), nums(1), nums(2)))
      } 
      catch {
        case _: NumberFormatException => None
      }
    } 
    else {
      None
    }
  }
}

def printSsn(ssn: String): Unit = {
  val description = ssn match { 
    case SocialSecurityNumber(x, y, z) => s"Upper: $x, Mid: $y, Lower: $z"
    case _                             => s"$ssn is not a valid SSN"
  }
  println(description)
}

printSsn("123-45-6789")
printSsn("123-asdf-6789")

## Varargs Pattern Matching

We can also match a variable number of arguments — for example, binding a name to the first few values of a `Vector`.

In [ ]:
def foo(xs: Vector[Int]): Boolean = xs match {
  case Vector(1, 2) => true
  case _ => false
}

println("foo:\n----")
println(foo(Vector(1, 2)))
println(foo(Vector(1, 2, 3)))
println(foo(Vector(2, 3, 4)))

def bar(xs: Vector[Int]): Boolean = xs match {
  case Vector(1, 2, _*) => true
  case _ => false
}

println("\nbar:\n----")
println(bar(Vector(1, 2)))
println(bar(Vector(1, 2, 3)))
println(bar(Vector(2, 3, 4)))

In [ ]:
def baz(ps: Vector[PhoneNumber]): Boolean = ps match {
  case Vector(PhoneNumber("215", n1), PhoneNumber("267", n2), _*) if n1 == n2 => true
  case _ => false
}

println(baz(Vector(PhoneNumber("215", "555-1212"), PhoneNumber("267", "555-1212"))))
println(baz(Vector(PhoneNumber("215", "555-1212"), PhoneNumber("267", "555-5555"))))

### Vararg Extractors

We can write extractors that support vararg pattern matching by implementing `unapplySeq` instead of `unapply`.

In [ ]:
object Primes {
  def unapplySeq(n: Int): Some[Seq[Int]] = {
    Some(factorize(n))
  }
  
  // Adapted from http://stackoverflow.com/a/30283151/547212
  def factorize(x: Int): List[Int] = {
    @annotation.tailrec
    def loop(x: Int, a: Int = 2, acc: List[Int] = Nil): List[Int] = {
      a*a > x match {
        case false if x % a == 0 => loop(x / a, a, a :: acc)
        case false => loop(x, a + 1, acc)
        case true => x :: acc
      }
    }
    loop(x).reverse
  }
}

def printDivByTwoAndThree(n: Int): Unit = n match { 
  case Primes(2, 3, _*) => println(s"$n divisible by both 2 and 3") 
  case Primes(_*) => println(s"$n NOT divisible by both 2 and 3")
}
printDivByTwoAndThree(42)
printDivByTwoAndThree(24)

## Pattern-ing

So far, every use of pattern matching we've seen has involved matching an expression against one or more cases via the `match` keyword. Scala allows pattern matching in a number of other ways though.

### Pattern matching during `val` assignment

In [ ]:
val SocialSecurityNumber(hi, med, low) = "123-45-6789"
println(s"High: $hi, Medium: $med, Low: $low")

// This will throw a MatchError
// val SocialSecurityNumber(hi, med, low) = "asdf"

val Some(x) = Option(1)
println(s"x: $x")

val y @ Some(z) = Option(42)
println(s"y: $y, z: $z")

Be careful with `val` assignment matching, though: If you get it wrong, you'll get an exception at runtime:

In [ ]:
val Array(first, second) = (1 to 3).toArray // this will bail

When destructuring something of unknown size, it's best to use the _varargs_ pattern or a full pattern match.

In [ ]:
val arr = (1 to 3).toArray
val (first, second) = arr match {
  case Array() => (0, 0)
  case Array(n) => (n, 0)
  case Array(m, n, _*) => (m, n)
}

### Pattern matching in the body of an anonymous function

This technique is a nice way to unpack an argument to a lambda:

In [ ]:
val firstNames = AllPeople.map { case Person(firstName, _, _, _, _) => firstName }

It is _especially_ useful with tuples. Compare these two examples. Both do exactly the same thing. Which one is more readable?


In [ ]:
val m = Seq("key1", "key2", "key3", "key4").map { key => key -> math.abs(scala.util.Random.nextInt(1000)) }

// Let's create a new map which doubles each value. Which do you like better?

val m1 = m.map { pair => pair._1 -> pair._2 * 2 }
val m2 = m.map { case (key, value) => key -> value * 2 }

### Pattern matching in the definition of a PartialFunction

This is the definition of a `PartialFunction` from `A` to `B`:

```scala
trait PartialFunction[-A,+B] {
  def isDefinedAt(a: A): Boolean
  def apply(a: A): B
}
```

This defines a function which is only safe to call at a given value of type `A` if the `isDefinedAt` method returns true for that value.

One way to think of a partial function is as an incomplete pattern matching expression:

In [ ]:
val pf1: PartialFunction[(String, Int), (String, Int)] = { case (key: String, value: Int) if value > 0 => key -> value * 2 }

Consider the `collect` method in the standard collections:

```scala
class Iterable[+A] extends ... { 
  /** Filters and maps in a single step. */
  def collect[B](pf: PartialFunction[A, B]): Iterable[B] = {
    // Note: toy implementation - real implementation is significantly more powerful and performant
    this.filter { a => pf.isDefinedAt(a) }.map { a => pf(a) }
  }
}
```

To use `collect`, we could manually instantiate a subtype of `PartialFunction`, as we did above.

But that's *far* more verbose than simply filtering and then mapping:

In [ ]:
val phoneNumbers = AllPeople.collect(new PartialFunction[Person, PhoneNumber] {
  def isDefinedAt(p: Person): Boolean = p.phoneNumber.isDefined
  def apply(p: Person): PhoneNumber = p.phoneNumber.get
})

But, as we just saw, a partial pattern match expression _compiles_ to a partial function, so this works:

In [ ]:
val phoneNumbers = AllPeople.collect { case Person(_, _, _, _, Some(ph)) => ph }

### Actors

Perhaps the most famous use of defining partial functions via pattern matching is from Akka:

In [ ]:
trait Actor {
  def receive: Actor.Receive
}

object Actor {
  type Receive = PartialFunction[Any, Unit]
}

case class SomeMessage(data: String, reply: Boolean)
case class AnotherMessage(data: String, reply: Boolean, expiry: java.time.Instant)

class MyActor extends Actor {
  def receive = {
    case SomeMessage(data, reply) => ???
    case AnotherMessage(data, reply, expiry) => ???
  }
}

# Data (Algebraic Data Types)

Earlier, we defined a case class as one in which a bunch of code was generated for us. This is a practical way to think about case classes, but limits us in certain ways.

Instead, let's reconsider a case class as a kind of *data*. More specifically, as a *labeled tuple*: a tuple with names attached to each value.

Contrast `(Int, Int)` with `case class Point(x: Int, y: Int)`:

 - Both types have the same number of elements (2)
 - Both types can store the same range of possible values ($2^{64}$)
 - There's a natural mapping between these types (`_1` &harr; `x`, `_2` &harr; `y`)
 - The case class has a label attached to each field (`x`, `y`)
 - The case class has a label attached to the class itself (`Point`)

We can think of a case class as a _product_ of some number of types — in the case of `Point`, an `Int` *and* `an `Int`.

For example, consider the product of a `Boolean`, `Boolean`, and `Int`:

In [ ]:
case class Woozle(foo: Boolean, bar: Boolean, count: Int)

This is a labeled version of the product of `(Boolean, Boolean, Int)` and it has $2 \cdot 2 \cdot 2^{32} = 2^{34}$ possible values.

Another way to think of this is that a value of the product type $A \cdot B \cdot C \cdot$... 
consists of a value for `A` and a value for `B` and a value of `C` and so on. 

We can then consider the dual to product types, where instead of having a type that consists of a value of `A` *and* a value of `B` *and* a value of `C` and so on, we replace the ANDs with ORs. We'll call these sum types for a reason that will become clear in a moment.

Let's informally write a sum type as $A + B + C +$..., where a value of this type is a value of `A` *or* a value of `B` *or* a value of `C` and so on. 

(**Technicality**: Note that just like in the product case, the component types of our sum type do not have to be unique. Our encoding in Scala must be able to handle this without losing track of the "position" of a value).

So why do we call this a "sum" type? Consider the number of values of the type $A + B + C +$...

- There's a value of the sum type for each value of `A`, each value of `B`, etc.
- The total number of values of the sum type is the number of values of `A` plus the number of values of `B` plus the number of values of `C`, etc.

#### Encoding sum types in Scala

We could use a standard library data type, `scala.util.Either[L, R]`, but that can get pretty inconvenient pretty quickly as we nest `Either`s. Further, it suffers from the same problem as tuples in that the components are *unlabeled*.

Instead, let's use subtyping! We've alread seen some very common examples, in fact.

```scala
sealed abstract class Option[+A] { ...}
case class Some[+A](get: A) extends Option[A]
case object None extends Option[Nothing]
```

Similarly, `List:`

```scala
sealed abstract class List[+A] { ... }
case class ::[+A](head: A, tail: List[A]) extends List[A]
case object Nil extends List[Nothing]
```

These are both sum types, though they feel a bit artificial since they both represent a choice between a singleton and non-singleton. Let's consider a domain specific example.

In [ ]:
sealed trait Taxable
case class Individual(firstName: String, lastName: String, ssn: String) extends Taxable
case class Business(name: String, taxIdNumber: String) extends Taxable

def taxId(t: Taxable): String = t match {
  case Individual(_, _, ssn) => ssn
  case Business(_, tin) => tin
}

As you can see, there's nothing new here — just standard pattern matching that we've used before. The differences are more about philosophy and design mindset. 

Instead of thinking about defining classes and methods, we instead:

- focus on defining *data*, using product and sum types
- and creating functions that operate on that data